# Посетитель (visitor)

Посетитель — это поведенческий паттерн проектирования, который позволяет добавлять в программу новые операции, не изменяя классы объектов, над которыми эти операции могут выполняться.

На вход дано дерево организационной структуры компании. Узлами дерева являются сотрудники двух типов: руководители и линейный персонал. Требуется добавить функционал сохранения этого дерева в файл, при этом реализация **не должна** содержаться внутри существующих узлов (таково требование архитектора ПО).

In [15]:
from typing import Iterable, List, Iterator
import abc


class AbsTree(abc.ABC):
    @abc.abstractproperty
    def name(self) -> str:
        ...

    @abc.abstractmethod
    def accept(self, visitor) -> None:
        """
        К готовой реализации нужно добавить только один метод.
        Он вызывает метод наблюдателя и передает текущий узел в качестве параметра.
        """
        ...


class Employee(AbsTree):
    def __init__(self, name: str) -> None:
        self.__name = name

    @property
    def name(self) -> str:
        return self.__name

    def accept(self, visitor) -> None:
        """
        Для каждого класса вызывается отдельный метод у наблюдателя.
        """
        visitor.visit_employee(self)


class Supervisor(Iterable, AbsTree):
    def __init__(self, name: str, members: List[AbsTree]) -> None:
        self.__name = name
        self.__members = members

    def __iter__(self) -> Iterator[AbsTree]:
        return iter(self.__members)

    @property
    def name(self) -> str:
        return self.__name

    def accept(self, visitor) -> None:
        """
        Пробегаемся по всем подчиненным руководителя.
        """
        visitor.visit_supervisor(self)
        for node in self:
            node.accept(visitor)

In [16]:
class AbsVisitor(abc.ABC):
    @abc.abstractmethod
    def visit_employee(self, employee: Employee) -> None:
        ...

    @abc.abstractmethod
    def visit_supervisor(self, supervisor: Supervisor) -> None:
        ...


class ExportToFileVisitor(AbsVisitor):
    """
    Оставляем логику сохранения в отдельном классе.
    """
    def visit_employee(self, employee: Employee) -> None:
        print("СОТРУДНИК:", employee.name)

    def visit_supervisor(self, supervisor: Supervisor) -> None:
        print("РУКОВОДИТЕЛЬ:", supervisor.name)
        print("ПОДЧИНЕННЫЕ:")

In [17]:
visitor = ExportToFileVisitor()

company = Supervisor(
    "Бровко",
    [
        Employee("Киреев"),
        Employee("Максимов"),
        Supervisor(
            "Каневский",
            [
                Employee("Рязанов"),
                Employee("Пигульский"),
            ],
        ),
    ],
)

company.accept(visitor)

РУКОВОДИТЕЛЬ: Бровко
ПОДЧИНЕННЫЕ:
СОТРУДНИК: Киреев
СОТРУДНИК: Максимов
РУКОВОДИТЕЛЬ: Каневский
ПОДЧИНЕННЫЕ:
СОТРУДНИК: Рязанов
СОТРУДНИК: Пигульский
